In [1]:
from gbm.finder import BurstCatalog
from gbm.finder import TriggerFtp
import os
    
import gbm
from gbm import test_data_dir
from gbm.data import TTE
from gbm.binning.unbinned import bin_by_time
import glob

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from gbm.plot import Lightcurve, Spectrum


burstcat = BurstCatalog()


# burstcat.num_rows


ModuleNotFoundError: No module named 'gbm'

In [ ]:
bursts = burstcat.get_table(columns=('name',
                                     'trigger_name', 
                                     't90',
                                     't90_start',
                                    ))

erange_nai = (8.0, 900.0) # in keV
erange_bgo = (325, 35000.0)
    
failed = []

# for i in range(len(bursts.trigger_name)):
for i in range(1000):
# for i in [0,1,2]:

    print(i,len(bursts.trigger_name),bursts.trigger_name[i])
    
    try:
        # initialize the Trigger data finder with a trigger number
        burst = bursts.trigger_name[i]
        trig_finder = TriggerFtp(burst[2:])

        #check if exists 
        path_tte = './Bursts/'+bursts.trigger_name[i]

        if not os.path.isdir(path_tte) or len(path_tte)<14 or not os.path.isfile(path_tte+'/done.txt'):
            trig_finder.get_tte(path_tte,verbose = False)

        else:
            print('TTE Downloaded')
            continue

        files_tte = glob.glob(path_tte+'/*tte*')

        temp_time = []
        temp_flux = []
        temp_idx = []
        temp_bg = []
        temp_spec = []
        temp_ebin = []
        temp_det = []

        for ii in range(len(files_tte)):
        # open a TTE file
            tte = TTE.open(files_tte[ii])
            # bin to 1.024 s resolution, reference time is trigger time
            phaii = tte.to_phaii(bin_by_time, 1.024*2, time_ref=0.0)

            # maybe change the energy range??

            erange = erange_nai
            if 'tte_b' in files_tte[ii]:
                erange = erange_bgo

            lc_data = phaii.to_lightcurve(energy_range=erange)

            # lcplot = Lightcurve(data=lc_data)
            bg = np.median(lc_data.counts)
            t90 = bursts.t90[i]
            t90_start = bursts.t90_start[i]

            # median and std of the t90 region
            cond_bg_t90 = (lc_data.centroids>(t90_start-15)) * (lc_data.centroids<(t90_start+t90+35))
            cond_t90 = (lc_data.centroids>(t90_start)) * (lc_data.centroids<(t90_start+t90))
            std_bg_t90 = np.std(lc_data.counts[cond_bg_t90])
            median_t90 = np.max(lc_data.counts[cond_t90])-np.median(lc_data.counts[cond_bg_t90])
            SNR = median_t90/std_bg_t90

            if SNR > 3:
                print(ii,median_t90,std_bg_t90,median_t90/std_bg_t90)
                temp_time.append(lc_data.centroids[1:-1])
                temp_flux.append(lc_data.counts[1:-1])
                temp_idx.append(ii)
                temp_bg.append(bg)
                temp_det.append(files_tte[ii][files_tte[ii].find('tte')+4:files_tte[ii].find('tte')+6])
                det = files_tte[ii][files_tte[ii].rfind('glg_tte_')+8:files_tte[ii].rfind('glg_tte_')+8+2]
                lc_name = f'lc_{burst}_{det}_{np.round(t90_start,1)}_{np.round(t90,1)}_{np.round(SNR,1)}.txt'
                np.savetxt(path_tte+'/'+lc_name,np.array([lc_data.centroids[1:-1],lc_data.counts[1:-1]]))

                # spectral data
                # our lightcurve source selection
                src_time = (t90_start, t90_start+t90)
                src_lc = phaii.to_lightcurve(time_range=src_time, energy_range=erange)
                # the spectum of the energy range selection 
                spec_selection = phaii.to_spectrum(time_range=src_time, energy_range=erange)
                c_erange = (spec_selection.centroids < max(erange)) * (spec_selection.centroids > min(erange))
                spec_name =  f'spec_{burst}_{det}_{np.round(t90_start,1)}_{np.round(t90,1)}_{np.round(SNR,1)}.txt'
                np.savetxt(path_tte+'/'+spec_name,np.array([spec_selection.centroids[c_erange],spec_selection.counts[c_erange]]))
                temp_ebin.append(spec_selection.centroids[c_erange])
                temp_spec.append(spec_selection.counts[c_erange])


        if len(temp_flux) == 0:
            print('No SNR > 3')
            failed.append(bursts.trigger_name[i])
#             if os.path.isfile(path_tte+'/done.txt'):
            cmd = f'rm {path_tte}/*tte*.fit'
            os.system(cmd)
            print(cmd)


        fig, axs = plt.subplots(len(temp_flux), 2, figsize=(7, 2*len(temp_flux)))
        for jj,ax in enumerate(axs.flat):
            j = int(jj/2)
            if jj%2 == 0: #even
                ax.set_title('LC det:'+ temp_det[j])
                ax.vlines(t90_start-15,temp_bg[j]-100,temp_bg[j]+100,color='red')
                ax.vlines(t90_start+t90+35,temp_bg[j]-100,temp_bg[j]+100,color='red')
                ax.vlines(t90_start,temp_bg[j]-100,temp_bg[j]+100)
                ax.vlines(t90_start+t90,temp_bg[j]-100,temp_bg[j]+100)
                ax.plot(temp_time[j],temp_flux[j])
            else:
                ax.set_title('spec det:'+ temp_det[j])
                ax.plot(temp_ebin[j],temp_spec[j])
                ax.set_xscale('log')
                ax.set_yscale('log')
        fig.tight_layout() 
        fig.savefig(path_tte+'/'+f'collage_{burst}.png',dpi=200)
        plt.close(fig)

        # done with data analysis
        np.savetxt(path_tte+'/'+'done.txt',np.array([]))

        if os.path.isfile(path_tte+'/done.txt'):
            cmd = f'rm {path_tte}/*tte*.fit'
            os.system(cmd)
            print(cmd)
    
    except:
        print('FAILED', bursts.trigger_name[i])
        failed.append(bursts.trigger_name[i])
#         if os.path.isfile(path_tte+'/done.txt'):
        cmd = f'rm {path_tte}/*tte*.fit'
        os.system(cmd)
        print(cmd)

